In [124]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Activation, Flatten, Input, BatchNormalization, Reshape, UpSampling2D

# Generator

In [125]:
def Generator(image_width,image_height,channel):
    # 入力は100次元のノイズ
    inputs = Input((100,))
    in_w = int(image_width / 4)
    in_h = int(image_height / 4)
    x = Dense(in_h * in_w * 128, activation='tanh', name='g_dense1')(inputs)
    x = BatchNormalization()(x)
    x = Reshape((in_h, in_w, 128), input_shape=(128 * in_h * in_w,))(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(64, (5, 5), padding='same', activation='tanh', name='g_conv1')(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(channel, (5, 5), padding='same', activation='tanh', name='g_out')(x)
    model = Model(inputs, x, name='Generator')
    return model

# Discriminator

In [126]:
def Discriminator(image_width,image_height,channel):
    inputs = Input((image_height, image_width, channel))
    x = Conv2D(64, (5, 5), padding='same', activation='tanh', name='d_conv1')(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (5, 5), padding='same', activation='tanh', name='d_conv2')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(1024, activation='relu', name='d_dense1')(x)
    x = Dense(1, activation='sigmoid', name='d_out')(x)
    model = Model(inputs, x, name='Discriminator')
    return model

# GAN

In [127]:
def GAN(generator,discriminator):
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Create Data

In [ ]:
def extract_train_data(data_source):
    

In [128]:
generator = Generator(image_height=32, image_width=32, channel=3)
discriminator = Discriminator(image_height=32, image_width=32, channel=3)
gan = GAN(generator=generator, discriminator=discriminator)

In [13]:
generator.summary()
discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 100)               0         
_________________________________________________________________
g_dense1 (Dense)             (None, 8192)              827392    
_________________________________________________________________
batch_normalization_2 (Batch (None, 8192)              32768     
_________________________________________________________________
reshape_2 (Reshape)          (None, 8, 8, 128)         0         
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, 16, 16, 128)       0         
_________________________________________________________________
g_conv1 (Conv2D)             (None, 16, 16, 64)        204864    
_________________________________________________________________
up_sampling2d_4 (UpSampling2 (None, 32, 32, 64)        0         
__________

In [129]:
g_opt = keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
d_opt = keras.optimizers.Adam(lr=0.0002, beta_1=0.5)

In [130]:
# Generatorの学習
# Generatorの学習時にはDiscriminatorの重みは更新しない
generator.compile(loss='binary_crossentropy', optimizer='SGD')
discriminator.trainable = False
for layer in discriminator.layers:
    layer.trainable = False
gan.compile(loss='binary_crossentropy', optimizer=g_opt)

In [131]:
# Discriminatorの学習
discriminator.trainable = True
for layer in discriminator.layers:
    layer.trainable = True
discriminator.compile(loss='binary_crossentropy', optimizer=d_opt)

In [200]:
# データセット(cifar10)を読み込む
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [214]:
# ネコ（３）と犬（５）だけのデータにする
cat_train_index = np.where(y_train == 3)[0]
dog_train_index = np.where(y_train == 5)[0]
catdog_y_train = np.concatenate([cat_train_index,dog_train_index])
dogcat_x_train = x_train[catdog_y_train]
dogcat_x_train = (dogcat_x_train.astype(np.float32) - 127.5)/127.5
cat_test_index = np.where(y_test == 3)[0]
dog_test_index = np.where(y_test == 5)[0]
catdog_y_test = np.concatenate([cat_test_index,dog_test_index])
dogcat_x_test = x_test[catdog_y_test]
train_num = dogcat_x_train.shape[0]
train_num_per_step = train_num // 64
print(train_num_per_step)

156


In [216]:
minibatch = 64
for ite in range(3000):
    ite += 1
    # Discremenator training
    train_ind = ite % (train_num_per_step - 1)
    y = dogcat_x_train[train_ind * minibatch : (train_ind+1) * minibatch]
    input_noise = np.random.uniform(-1, 1, size=(64, 100))
    g_output = generator.predict(input_noise, verbose=0)
    #print(y.shape)
    #print(g_output.shape)
    X = np.concatenate((y, g_output))
    #print(X.shape)
    Y = [1] * 64 + [0] * 64
    # print(Y)
    d_loss = discriminator.train_on_batch(X, Y)
    # Generator training
    input_noise = np.random.uniform(-1, 1, size=(minibatch, 100))
    g_loss = generator.train_on_batch(input_noise, [1] * minibatch)

(64, 32, 32, 3)
(64, 32, 32, 3)
(128, 32, 32, 3)
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


ValueError: Error when checking target: expected g_out to have 4 dimensions, but got array with shape (64, 1)

In [205]:
dogcat_x_train.shape

(10000, 32, 32, 3)